# Initialization

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DataType
from pyspark.sql.functions import trim, col

## Read Bronze Table

In [0]:
df = spark.table("workspace.bronze.erp_cust_az12")

# Silver transformation

## Trimming

In [0]:
for field in df.schema.fields:
  if isinstance(field.dataType, StringType):
    df = df.withColumn(field.name, trim(col(field.name)))

## Customer Id Cleanup

In [0]:
df = df.withColumn(
    "cid",
    F.when(col("cid").startswith("NAS"),
           F.substring(col("cid"), 4, F.length(col("cid"))))
    .otherwise(col("cid"))
)

## Birth Data Validation

In [0]:
df = df.withColumn(
    "bdate",
    F.when(F.col("bdate") > F.current_date(), None).
    otherwise(F.col("bdate"))
)

## Gender Normalization

In [0]:
df = df.withColumn(
    "gen",
    F.when(F.upper(col("gen")).isin("F", "FEMALE"), "FEMALE").
    when(F.upper(col("gen")).isin("M", "MALE"), "MALE").
    otherwise("n/a")
)

## Renaming Column

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "bdate": "birth_date",
    "gen": "gender"
}
for old_name, new_name in RENAME_MAP.items():
    df.withColumnRenamed(old_name, new_name)

## Sanity checks of Dataframe

In [0]:
df.limit(10).display()

# Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customers")

## Sanity checks of silver table

In [0]:
%sql
SELECT * FROM workspace.silver.erp_customers LIMIT 10